# __Movie Sentiment Analysis__

In [1186]:
import pandas as pd
import numpy as np
import re
import contractions
import nltk
import contractions
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import text_preprocessing as tp

from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from flashtext import KeywordProcessor


warnings.filterwarnings("ignore")

## Importing Dataset for Movie Metadata

Use the IMDB dataset for movie details. Sources of the data are listed below.

__a. Data Source:__ https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/

In [1187]:
# Open file containing movie metadata
movies_metadata_df = pd.read_csv("./dataset/movies_metadata.csv")
movies_metadata_df.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,10/30/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,12/15/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,12/22/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,12/22/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,2/10/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [1188]:
# Open file containing the movie casts and the crews
movies_credits_df = pd.read_csv("./dataset/credits.csv")
movies_credits_df.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [1189]:
# Combine the dataframes of movie metadata and credits
movie_metadata_credits_df = pd.merge(movies_metadata_df, movies_credits_df, on=["id"], how="left")
movie_metadata_credits_df.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [1190]:
# List all the columns (to show to hidden columns)
movie_metadata_credits_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [1191]:
# Drop irrelevant columns
del_columns = ['belongs_to_collection', 'homepage', 'id', 'original_language', 'original_title', 'popularity', 'poster_path', 'overview',
               'tagline', 'production_countries', 'runtime', 'spoken_languages', 'status', 'video', 'vote_average', 'vote_count']
movie_metadata_credits_df = movie_metadata_credits_df.drop(del_columns, axis=1)

# Remove 'tt' appended on the IMDB ID
movie_metadata_credits_df['imdb_id'] = movie_metadata_credits_df['imdb_id'].apply(lambda x : x[2:] if isinstance(x, str) and "tt" in x else x)

movie_metadata_credits_df.head(5)

,adult,budget,genres,imdb_id,production_companies,release_date,revenue,title,cast,crew
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",0114709,"[{'name': 'Pixar Animation Studios', 'id': 3}]",10/30/1995,373554033.0,Toy Story,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",0113497,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",12/15/1995,262797249.0,Jumanji,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0113228,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",12/22/1995,0.0,Grumpier Old Men,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",0114885,[{'name': 'Twentieth Century Fox Film Corporat...,12/22/1995,81452156.0,Waiting to Exhale,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",0113041,"[{'name': 'Sandollar Productions', 'id': 5842}...",2/10/1995,76578911.0,Father of the Bride Part II,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


__b. Data Source:__ https://www.kaggle.com/datasets/raynardj/imdb-vision-and-nlp/data

In [1192]:
# Open another file containing the movie metadata
movies_df = pd.read_csv("./dataset/movies.csv")
movies_df.head(5)

,imdbId,Imdb Link,Title,IMDB Score,Genre,local_image_path
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation;Adventure;Comedy,images/114709_.jpg
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action;Adventure;Family,images/113497_.jpg
2,113277,http://www.imdb.com/title/tt113277,Heat (1995),8.2,Action;Crime;Drama,images/113277_.jpg
3,114319,http://www.imdb.com/title/tt114319,Sabrina (1995),6.3,Comedy;Drama,images/114319_.jpg
4,114576,http://www.imdb.com/title/tt114576,Sudden Death (1995),5.7,Action;Crime;Thriller,images/114576_.jpg


In [1193]:
# Rename the column names of movies info by removing the spaces and converting to lower case
movies_df = movies_df.rename(columns={"imdbId" : "imdb_id", "Imdb Link" : "imbd_link", "Title" : "movie", "IMDB Score" : "imdb_score", "Genre" : "genre"})
print(movies_df.columns)

Index(['imdb_id', 'imbd_link', 'movie', 'imdb_score', 'genre',
       'local_image_path'],
      dtype='object')


In [1194]:
# Drop irrelevant columns
del_columns = ['imbd_link', 'imdb_score', 'local_image_path']

movies_df = movies_df.drop(columns=del_columns, axis=1)
movies_df.head(5)

,imdb_id,movie,genre
0,114709,Toy Story (1995),Animation;Adventure;Comedy
1,113497,Jumanji (1995),Action;Adventure;Family
2,113277,Heat (1995),Action;Crime;Drama
3,114319,Sabrina (1995),Comedy;Drama
4,114576,Sudden Death (1995),Action;Crime;Thriller


In [1195]:
# Merge two different datasets for movies metadata 

movie_metadata_credits_df['imdb_id'] = movie_metadata_credits_df['imdb_id'].fillna(0) # to fix errors on NaN values when converting to integer
movie_metadata_credits_df['imdb_id'] = movie_metadata_credits_df['imdb_id'].astype(int)

movies_metadata_combined_df = pd.merge(movie_metadata_credits_df, movies_df, on=["imdb_id"], how="right")
movies_metadata_combined_df.head(5)

,adult,budget,genres,imdb_id,production_companies,release_date,revenue,title,cast,crew,movie,genre
0,False,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",114709,"[{'name': 'Pixar Animation Studios', 'id': 3}]",10/30/1995,373554033.0,Toy Story,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",Toy Story (1995),Animation;Adventure;Comedy
1,False,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",113497,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",12/15/1995,262797249.0,Jumanji,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",Jumanji (1995),Action;Adventure;Family
2,False,60000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",113277,"[{'name': 'Regency Enterprises', 'id': 508}, {...",12/15/1995,187436818.0,Heat,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",Heat (1995),Action;Crime;Drama
3,False,58000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",114319,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",12/15/1995,0.0,Sabrina,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",Sabrina (1995),Comedy;Drama
4,False,35000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",114576,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",12/22/1995,64350171.0,Sudden Death,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",Sudden Death (1995),Action;Crime;Thriller


## Importing Dataset for Movie Reviews

Use the IMDB dataset for movie review. Source of the data is listed below.

__Data Source:__ https://www.kaggle.com/datasets/raynardj/imdb-vision-and-nlp/data

In [1196]:
# Open the file containing the movie reviews
reviews_df = pd.read_csv("./dataset/reviews.csv")
reviews_df.head(5)

,review_id,reviewer,movie,rating,review_summary,review_date,spoiler_tag,review_detail,helpful
0,rw1133942,OriginalMovieBuff21,Kill Bill: Vol. 2 (2004),8.0,Good follow up that answers all the questions,24 July 2005,0,"After seeing Tarantino's Kill Bill Vol: 1, I g...","['0', '1']"
1,rw1133959,lost-in-limbo,Feardotcom (2002),3.0,"""I couldn't make much sense of it myself"". Too...",24 July 2005,0,There's a Website called FearDotCom and anyone...,"['1', '4']"
2,rw1133985,NateManD,Persona (1966),10.0,Persona gives me all the reasons to love art-h...,24 July 2005,0,"Long before ""Muholland Drive"" there was anothe...","['9', '23']"
3,rw1133999,CAMACHO-4,War of the Worlds (2005),3.0,A disappointing film from the team that you Mi...,24 July 2005,0,Spielberg said this film is based on the H.G. ...,"['9', '14']"
4,rw1134010,CAMACHO-4,Mr. & Mrs. Smith (2005),6.0,A fun action movie with great chemistry,24 July 2005,0,"Director Doug Liman, who's gotten famous for m...","['1', '3']"


In [1197]:
# Drop columns that are not relevant
del_columns = ['review_id', 'reviewer', 'spoiler_tag', 'helpful', 'rating']

movie_reviews_df = movie_reviews_df.drop(columns=del_columns, axis=1)
movie_reviews_df.head(5)

,imdb_id,movie,genre,review_summary,review_date,review_detail
0,114709,Toy Story (1995),Animation;Adventure;Comedy,"""Toy Story"" - The Toys are in town and somethi...",6 September 2005,"I'm 19, going on 20 in six days. I don't need ..."
1,114709,Toy Story (1995),Animation;Adventure;Comedy,Pixar's first full length movie.,1 October 2005,This was the first Pixar movie I've seen. I re...
2,114709,Toy Story (1995),Animation;Adventure;Comedy,Classic Pixar! This is what future Pixar films...,26 October 2005,Remember the original Pixar movies? Remember w...
3,114709,Toy Story (1995),Animation;Adventure;Comedy,One of the first animations from Pixar,10 November 2005,"When Toy Story came, in 1995,I was 9 years old..."
4,114709,Toy Story (1995),Animation;Adventure;Comedy,Can it be ten years already?,23 November 2005,PIXAR animation was but an experimental animat...


In [1198]:
# Merge the movie metadata (from same source) and reviews in single dataframe
movie_reviews_df = pd.merge(movies_df, reviews_df, on=["movie"], how="left")
movie_reviews_df.head(5)

,imdb_id,movie,genre,review_id,reviewer,rating,review_summary,review_date,spoiler_tag,review_detail,helpful
0,114709,Toy Story (1995),Animation;Adventure;Comedy,rw1166979,dee.reid,10.0,"""Toy Story"" - The Toys are in town and somethi...",6 September 2005,0,"I'm 19, going on 20 in six days. I don't need ...","['5', '5']"
1,114709,Toy Story (1995),Animation;Adventure;Comedy,rw1184497,emasterslake,NaN,Pixar's first full length movie.,1 October 2005,0,This was the first Pixar movie I've seen. I re...,"['0', '0']"
2,114709,Toy Story (1995),Animation;Adventure;Comedy,rw1202306,patrick_dunne,10.0,Classic Pixar! This is what future Pixar films...,26 October 2005,0,Remember the original Pixar movies? Remember w...,"['0', '0']"
3,114709,Toy Story (1995),Animation;Adventure;Comedy,rw1213162,Lady_Targaryen,8.0,One of the first animations from Pixar,10 November 2005,0,"When Toy Story came, in 1995,I was 9 years old...","['27', '43']"
4,114709,Toy Story (1995),Animation;Adventure;Comedy,rw1223346,KUAlum26,10.0,Can it be ten years already?,23 November 2005,0,PIXAR animation was but an experimental animat...,"['1', '1']"


## Exploratory Data Analysis

Check and resolve duplicate records

In [1199]:
# Drop duplicate records in movies metadata
print(f"BEFORE: Record count => {movies_metadata_combined_df.shape[0]}")
movies_metadata_combined_df.drop_duplicates(inplace=True)
print(f"AFTER: Record count => {movies_metadata_combined_df.shape[0]}")

BEFORE: Record count => 4174
AFTER: Record count => 4071


In [1200]:
# Drop duplicate records in movies reviews
print(f"BEFORE: Record count => {movie_reviews_df.shape[0]}")
movie_reviews_df.drop_duplicates(inplace=True)
print(f"AFTER: Record count => {movie_reviews_df.shape[0]}")

BEFORE: Record count => 557581
AFTER: Record count => 542637


Check and resolve missing values

In [1201]:
# Check for missing values in movies metadata
movies_metadata_combined_df.isna().sum()

adult                   3
budget                  3
genres                  3
imdb_id                 0
production_companies    3
release_date            3
revenue                 3
title                   3
cast                    3
crew                    3
movie                   0
genre                   0
dtype: int64

In [1202]:
# List of movies with missing information
movies_metadata_combined_df[movies_metadata_combined_df['title'].isna()]

,adult,budget,genres,imdb_id,production_companies,release_date,revenue,title,cast,crew,movie,genre
2780,NaN,NaN,NaN,462322,NaN,NaN,NaN,NaN,NaN,NaN,Grindhouse (2007),Action;Horror;Thriller
3521,NaN,NaN,NaN,448694,NaN,NaN,NaN,NaN,NaN,NaN,Puss in Boots (2011),Animation;Action;Adventure
3793,NaN,NaN,NaN,1937390,NaN,NaN,NaN,NaN,NaN,NaN,Nymphomaniac: Vol. I (2013),Drama


In [1203]:
# Drop all movies with missing information
movies_metadata_combined_df.dropna(inplace=True, axis=0)
movies_metadata_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4068 entries, 0 to 4173
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 4068 non-null   object 
 1   budget                4068 non-null   float64
 2   genres                4068 non-null   object 
 3   imdb_id               4068 non-null   int64  
 4   production_companies  4068 non-null   object 
 5   release_date          4068 non-null   object 
 6   revenue               4068 non-null   float64
 7   title                 4068 non-null   object 
 8   cast                  4068 non-null   object 
 9   crew                  4068 non-null   object 
 10  movie                 4068 non-null   object 
 11  genre                 4068 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 413.2+ KB


In [1206]:
# Show statistical summary for numerical values
movies_metadata_combined_df.describe()

,budget,imdb_id,revenue
count,4.068000e+03,4.068000e+03,4.068000e+03
mean,2.998213e+07,5.755249e+05,9.186798e+07
std,4.280689e+07,8.030515e+05,1.726654e+08
min,0.000000e+00,4.170000e+02,0.000000e+00
25%,3.780000e+05,8.898075e+04,2.065425e+05
50%,1.300000e+07,2.129305e+05,2.830403e+07
75%,4.000000e+07,8.044700e+05,1.030304e+08
max,3.800000e+08,5.247022e+06,2.787965e+09


In [1204]:
# Check for missing values in movies reviews
movie_reviews_df.isna().sum()

imdb_id               0
movie                 0
genre                 0
review_id             0
reviewer              0
rating            76870
review_summary        1
review_date           0
spoiler_tag           0
review_detail         1
helpful               0
dtype: int64

In [1205]:
# Replace all missing values in review summary and detail with a blank
movie_reviews_df[['review_summary', 'review_detail']] = movie_reviews_df[['review_summary', 'review_detail']].fillna('')
movie_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 542637 entries, 0 to 557580
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   imdb_id         542637 non-null  int64  
 1   movie           542637 non-null  object 
 2   genre           542637 non-null  object 
 3   review_id       542637 non-null  object 
 4   reviewer        542637 non-null  object 
 5   rating          465767 non-null  float64
 6   review_summary  542637 non-null  object 
 7   review_date     542637 non-null  object 
 8   spoiler_tag     542637 non-null  int64  
 9   review_detail   542637 non-null  object 
 10  helpful         542637 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 49.7+ MB


__Data Wrangling:__ Conduct data wrangling on movies metadata to transform and structure data into a desired format, making it useful for further analysis

In [1207]:
# Convert genre into list
# movie_reviews_df['genre'] = movie_reviews_df['genre'].apply(lambda x : x.split(";"))
# movie_reviews_df['genre'].head(5)

In [1208]:
# Separate movie release year from the title
# pattern_year = r'\((\d{4})\)$'

# movie_reviews_df['release_year'] = movie_reviews_df['movie'].str.extract(pattern_year).astype(int)
# movie_reviews_df['movie'] = movie_reviews_df['movie'].str.replace(pattern_year, '', regex=True).str.strip()
# movie_reviews_df[['movie', 'release_year']].head(5)

__For Text Pre-processing__ 
- Remove non-grammatical text like emails and URLs
- Replace emojis with English word/s
- Handle contractions

In [1209]:
emoji_dict = tp.get_emojis()
slang_word_dict = tp.webscrape_slang_words()

def initial_text_preprocessing(text):
    try:
        # Remove non-grammatical text
        text = tp.remove_email_address(text)
        text = tp.remove_hyperlink(text)

        # Replace emojis with English word/s
        text = emoji_dict.replace_keywords(text)

        # Handle contractions
        text = text.replace('İ', 'I')   # to handle errors thrown when fixing the contractions
        text = contractions.fix(text)

        # Replace slang words
        text = slang_word_dict.replace_keywords(text)

    except Exception as err:
        print(f"ERROR: {err}")
        print(f"Input Text: {text}")

    return text


text_prep_columns = ['movie', 'review_summary', 'review_detail']

movie_reviews_df[text_prep_columns] = movie_reviews_df[text_prep_columns].applymap(initial_text_preprocessing)
movie_reviews_df[text_prep_columns]

## Model Creation

## Model Evaluation

## References
- https://medium.com/analytics-vidhya/text-preprocessing-nlp-basics-430d54016048
- https://towardsdatascience.com/how-to-preprocess-social-media-data-and-text-messages-b011efacf74
- https://towardsdatascience.com/5-lesser-known-python-libraries-for-your-next-nlp-project-ff13fc652553
- https://medium.com/coinmonks/remaking-of-shortened-sms-tweet-post-slangs-and-word-contraction-into-sentences-nlp-7bd1bbc6fcff
- https://sunscrapers.com/blog/9-best-python-natural-language-processing-nlp/